# RAG

Retrieval-Augmented Generation (RAG) is a technique that combines the strengths of pre-trained language models with the ability to retrieve information from a large corpus of documents. RAG **enables the language model to produce more informed, accurate, and contextually relevant answers** than by relying on its pre-trained knowledge alone.

At Cohere, all RAG calls come with... **precise citations**! 🎉
The model cites which groups of words, in the RAG chunks, were used to generate the final answer.  
These citations make it easy to check where the model’s generated response claims are coming from and they help users gain visibility into the model reasoning.  

RAG consists of 3 steps:
- Step 1: Indexing and given a user query, retrieve the relevant chunks from the index
- Step 2: Optionally, rerank the retrieved chunks
- Step 3: Generate the model final answer with **precise citations**, given the retrieved and reranked chunks




## Step 0 - Imports & Getting some data

In this example, we'll use a recent piece of text, that wasn't in the training data: the Wikipedia page of the movie "Dune 2".   

In practice, you would typically do RAG on much longer text, that doesn't fit in the context window of the model.

In [2]:
# we'll use Cohere to cover all building blocks of RAG
# TODO: upgrade to "cohere>5"
%pip install "cohere<5" --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.9/52.9 kB 900.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.9 MB/s eta 0:00:00


In [1]:
!git clone https://github.com/sun35/cohere-build-day

Cloning into 'cohere-build-day'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 0), reused 9 (delta 0), pack-reused 0
Receiving objects: 100% (12/12), 8.61 KiB | 2.15 MiB/s, done.


In [4]:
import cohere
API_KEY = "S8aNMuKrtYSlJtNvqQGhHJlV36lr4qR8sC0lJxAp" # fill in your Cohere API key here
co = cohere.Client(API_KEY)

In [5]:
# we'll get some wikipedia data
!pip install wikipedia --quiet
import wikipedia

  Preparing metadata (setup.py) ... done


In [6]:
# let's get the wikipedia article about Dune Part Two
article = wikipedia.page('Dune Part Two')
text = article.content
print(f"The text has roughly {len(text.split())} words.")

The text has roughly 5748 words.


In [18]:
!pip install trafilatura

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 13.1 MB/s eta 0:00:00


In [23]:
from tqdm import tqdm
from trafilatura.sitemaps import sitemap_search
from trafilatura import extract_metadata
import requests
from bs4 import BeautifulSoup

def get_urls_from_site(resource_url: str) -> list:
    # returns urls from a given site
    urls = sitemap_search(resource_url)
    return urls

def create_dataset(list_of_sites):
    print(list_of_sites)
    data = []
    for website in tqdm(list_of_sites, desc="Websites"):
        urls = get_urls_from_site(website)
        for url in tqdm(urls, desc="URLs"):
            try:
                response = requests.get(url)
                response.raise_for_status() #checks for success
                soup = BeautifulSoup(response.content, "html.parser")
                metadata = extract_metadata(response.content)
                title = soup.title.string
                description = metadata.description
                paragraphs = [p.get_text(strip=True) for p in soup.find_all("p")]
                content = "\n".join(paragraphs)
                d = {
                    "url": url,
                    "title": title,
                    "body": content,
                    "description": description,
                }
                data.append(d)
            except requests.exceptions.HTTPError as exceph:
                print(exceph)
            except requests.exceptions.ConnectionError as errc:
                print(errc)
            except requests.exceptions.Timeout as errt:
                print(errt)
            except requests.RequestException as err:
                print(err)
    print(data)
    print("data printed")
    return data

def scrape(list_of_sites: list) -> None:
    data = create_dataset(list_of_sites)
    print("created dataset")
    return data


In [24]:
dataset = scrape(get_urls_from_site('https://cohere.com/'))

['https://cohere.com/responsibility', 'https://cohere.com/research', 'https://cohere.com/deployment-options/oracle', 'https://cohere.com/newsroom', 'https://cohere.com/pricing', 'https://cohere.com/deployment-options/aws', 'https://cohere.com/deployment-options/amazon-bedrock', 'https://cohere.com/customer-stories/liveperson', 'https://cohere.com/privacy', 'https://cohere.com/careers', 'https://cohere.com/about', 'https://cohere.com/newsroom/articles', 'https://cohere.com/fine-tuning', 'https://cohere.com/saas-agreement', 'https://cohere.com/customer-stories/bluedot', 'https://cohere.com/security', 'https://cohere.com/business', 'https://cohere.com/generate', 'https://cohere.com/slo', 'https://cohere.com/deployment-options/amazon-sagemaker', 'https://cohere.com/events', 'https://cohere.com/research/aya', 'https://cohere.com/embeddings', 'https://cohere.com/search', 'https://cohere.com/startup-program-application', 'https://cohere.com/customer-stories/hyperwrite', 'https://cohere.com/bl

Websites: 100%|██████████| 46/46 [02:16<00:00,  2.97s/it]

[{'url': 'https://cohere.com/responsibility', 'title': 'Responsibility - Developing Safer Language Models | Cohere', 'body': "Introducing Command R+: Our new, most powerful model in the Command R family.\nTry now\nLarge language models help us build powerful systems that augment human capabilities, but they can also be used in ways that cause harm. We believe language model providers must develop their technologies responsibly. This means proactively working to build safer products and accepting a duty of care to users, the environment, and society.\nWe’ve invested in technical and non-technical measures to mitigate potential harm and make our development processes transparent. We've also established an advisory Responsibility Council empowered to inform our product and business decisions.\nWe are excited about the potential for algorithmic language understanding to improve accessibility, enable human computer interaction, and allow for broader human-to-human dialogue. If you want to u

In [25]:
dataset

[{'url': 'https://cohere.com/responsibility',
  'title': 'Responsibility - Developing Safer Language Models | Cohere',
  'body': "Introducing Command R+: Our new, most powerful model in the Command R family.\nTry now\nLarge language models help us build powerful systems that augment human capabilities, but they can also be used in ways that cause harm. We believe language model providers must develop their technologies responsibly. This means proactively working to build safer products and accepting a duty of care to users, the environment, and society.\nWe’ve invested in technical and non-technical measures to mitigate potential harm and make our development processes transparent. We've also established an advisory Responsibility Council empowered to inform our product and business decisions.\nWe are excited about the potential for algorithmic language understanding to improve accessibility, enable human computer interaction, and allow for broader human-to-human dialogue. If you want 

## Step 1 - Indexing and given a user query, retrieve the relevant chunks from the index

We index the document in a vector database. This requires getting the documents, chunking them, embedding, and indexing them in a vector database. Then we retrieved relevant results based on the users' query.

### We split the document into chunks of roughly 512 words

In [26]:
# For chunking let's use langchain to help us split the text
%pip install -qU langchain-text-splitters --quiet
from langchain_text_splitters import RecursiveCharacterTextSplitter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 9.9 MB/s eta 0:00:00


In [31]:
text = [doc['body'] for doc in dataset]

In [32]:
text

["Introducing Command R+: Our new, most powerful model in the Command R family.\nTry now\nLarge language models help us build powerful systems that augment human capabilities, but they can also be used in ways that cause harm. We believe language model providers must develop their technologies responsibly. This means proactively working to build safer products and accepting a duty of care to users, the environment, and society.\nWe’ve invested in technical and non-technical measures to mitigate potential harm and make our development processes transparent. We've also established an advisory Responsibility Council empowered to inform our product and business decisions.\nWe are excited about the potential for algorithmic language understanding to improve accessibility, enable human computer interaction, and allow for broader human-to-human dialogue. If you want to use our API to help create a better world, or stay informed of new developments, let us know.\nModel the worldas we hope it w

In [33]:
# Create basic configurations to chunk the text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

# Split the text into chunks with some overlap
chunks_ = text_splitter.create_documents(text)
chunks = [c.page_content for c in chunks_]
print(f"The text has been broken down in {len(chunks)} chunks.")

The text has been broken down in 519 chunks.


### Embed every text chunk

Cohere embeddings are state-of-the-art.

In [34]:
# Because the texts being embedded are the chunks we are searching over, we set the input type as search_doc
model="embed-english-v3.0"
response = co.embed(
    texts= chunks,
    model=model,
    input_type="search_document",
    embedding_types=['float']
)
embeddings = response.embeddings.float
print(f"We just computed {len(embeddings)} embeddings.")

We just computed 519 embeddings.


### Store the embeddings in a vector database

We use the simplest vector database ever: a python dictionary using `np.array()`.

In [35]:
# We use the simplest vector database ever: a python dictionary
!pip install numpy --quiet

In [36]:
import numpy as np
vector_database = {i: np.array(embedding) for i, embedding in enumerate(embeddings)}
# { 0: array([...]), 1: array([...]), 2: array([...]), ..., 10: array([...]) }

## Given a user query, retrieve the relevant chunks from the vector database


### Define the user question

In [37]:
query = "Tell me more about Command-R+?"

# Note: the relevant passage in the wikipedia page we're looking for is:
# "[...] Dune: Part Two was originally scheduled to be released on October 20, 2023, but was delayed to November 17, 2023, before moving forward two weeks to November 3, 2023, to adjust to changes in release schedules from other studios. It was later postponed by over four months to March 15, 2024, due to the 2023 Hollywood labor disputes. After the strikes were resolved, the film moved once more up two weeks to March 1, 2024. [...]"

### Embed the user question

Cohere embeddings are state-of-the-art.

In [38]:
# Because the text being embedded is the search query, we set the input type as search_query
response = co.embed(
    texts=[query],
    model=model,
    input_type="search_query",
    embedding_types=['float']
)
query_embedding = response.embeddings.float[0]
print("query_embedding: ", query_embedding)

query_embedding:  [-0.010910034, -0.038116455, -0.001168251, -0.01576233, -0.035583496, 0.004337311, -0.009490967, 0.026977539, -0.011604309, 0.0057411194, 0.017532349, 0.007888794, -0.014564514, -0.047088623, 0.05429077, -0.015106201, 0.028823853, -0.069885254, -0.07019043, -0.029266357, 0.047698975, -0.006580353, -0.041748047, 0.044158936, 0.0027923584, 0.033447266, 0.015113831, -0.028366089, -0.028167725, -0.032440186, -0.020065308, 0.028305054, 0.031921387, 0.030593872, -0.04547119, 0.034820557, -0.043151855, 0.020401001, -0.006252289, -0.0043144226, -0.043182373, 0.03010559, 0.020904541, -0.0871582, -0.037200928, 0.037384033, -0.072143555, -0.0071792603, 0.03704834, 0.03894043, -0.027359009, 0.025650024, -0.04055786, 0.042663574, -0.012954712, -0.012924194, -0.03100586, -0.010917664, 0.023132324, 0.0061035156, -0.0025501251, -0.0152282715, 0.018051147, 0.028396606, 0.035095215, 0.016738892, 0.039794922, 0.019821167, -0.018218994, -0.012741089, -0.032836914, -0.024108887, -0.020584

### Retrieve the most relevant chunks from the vector database

We use cosine similarity to find the most similar chunks

In [39]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Calculate similarity between the user question & each chunk
similarities = [cosine_similarity(query_embedding, chunk) for chunk in embeddings]
print("similarity scores: ", similarities)

# Get indices of the top 10 most similar chunks
sorted_indices = np.argsort(similarities)[::-1]

# Keep only the top 10 indices
top_indices = sorted_indices[:10]
print("Here are the indices of the top 10 chunks after retrieval: ", top_indices)

# Retrieve the top 10 most similar chunks
top_chunks_after_retrieval = [chunks[i] for i in top_indices]
print("Here are the top 10 chunks after retrieval: ")
for t in top_chunks_after_retrieval:
    print("== " + t)

similarity scores:  [0.5428507036779695, 0.14997770957277098, 0.16867896826511625, 0.10645847376646737, 0.08811095359784442, 0.06789713540835401, 0.4611787125103466, 0.17514820831507888, 0.16794335319030093, 0.1549749594301717, 0.15940989641649952, 0.20192713995188624, 0.12900615359951292, 0.19477198776075036, 0.13857694040186092, 0.1924313891301107, 0.19510638287570733, 0.13939429327892547, 0.17284737882663037, 0.1750701737100549, 0.12954659222430914, 0.12499862450122091, 0.20435642541289006, 0.19964455022487348, 0.18252880279449632, 0.17520809169507964, 0.39956978466187537, 0.14533661890678992, 0.20073077445313497, 0.15455521056973867, 0.14003316849468692, 0.1926193787620853, 0.10978431035809309, 0.08426536241229014, 0.15124646056228047, 0.4876079001655865, 0.10289315396198473, 0.07011622225396023, 0.09129280695685739, 0.449591430639063, 0.12769344253930673, 0.08800037669120331, 0.08497103619726788, 0.09219683674230587, 0.4173484644552343, 0.04608363377148057, 0.5561836600713004, 0.5

## Step 2 - Rerank the chunks retrieved from the vector database

We rerank the 10 chunks retrieved from the vector database. Reranking boosts retrieval accuracy.

Reranking lets us go from 10 chunks retrieved from the vector database, to the 3 most relevant chunks.

In [40]:
response = co.rerank(
    query=query,
    documents=top_chunks_after_retrieval,
    top_n=3,
    model="rerank-english-v2.0",
)

top_chunks_after_rerank = [result.document['text'] for result in response]
print("Here are the top 3 chunks after rerank: ")
for t in top_chunks_after_rerank:
    print("== " + t)

Here are the top 3 chunks after rerank: 
== Introducing Command R+: Our new, most powerful model in the Command R family.
Try now
Access our models directly through our API and pay for what you use, or deploy on Amazon SageMaker.
Get free, rate-limited usage for learning and prototyping. Usage is free until you go into production
Power your business with hosted language models.
Features
Default Model
Input
Output
Command R+
Input
$3.00
/1M tokens
Output
$15.00
/1M tokens
Default Model
Input
Output
Command R
Input
$0.50
/1M tokens
Output
$1.50
== Introducing Command R+: Our new, most powerful model in the Command R family.
Try now
We’re providing enhanced assistance to ambitious startups looking to put language AI at the heart of what they do.
== Introducing Command R+: Our new, most powerful model in the Command R family.
Try now
rerank
Rerank provides a powerful semantic boost to the search quality of any keyword or vector search system without requiring any overhaul or replacement.
r

## Step 3 - Generate the model final answer, given the retrieved and reranked chunks

In [41]:
# preamble containing instructions about the task and the desired style for the output.
preamble = """
## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

In [42]:
# retrieved documents
documents = [
    {"title": "chunk 0", "snippet": top_chunks_after_rerank[0]},
    {"title": "chunk 1", "snippet": top_chunks_after_rerank[1]},
    {"title": "chunk 2", "snippet": top_chunks_after_rerank[2]},
  ]

# get model response
response = co.chat(
  message=query,
  documents=documents,
  preamble=preamble,
  model="command-r",
  temperature=0.3
)

print("Final answer:")
print(response.text)

Final answer:
Command R+ is the most powerful model in the Command R family. It provides enhanced assistance to startups looking to implement language AI in their operations.

Compared to the default model, it has a higher input cost of $3.00/1M tokens and a higher output cost of $15.00/1M tokens.

Command R+ also has a feature called rerank. It boosts the search quality of any keyword or vector search system without requiring any system overhaul or replacement.


Note: this is indeed the answer you'd expect, and here was the passage of text in wikipedia explaining it!

" [...] Dune: Part Two was originally scheduled to be released on October 20, 2023, but was delayed to November 17, 2023, before moving forward two weeks to November 3, 2023, to adjust to changes in release schedules from other studios. It was later postponed by over four months to March 15, 2024, due to the 2023 Hollywood labor disputes. After the strikes were resolved, the film moved once more up two weeks to March 1, 2024. [...]"

## Bonus: Citations come for free with Cohere! 🎉

At Cohere, all RAG calls come with... precise citations! 🎉
The model cites which groups of words, in the RAG chunks, were used to generate the final answer.  
These citations make it easy to check where the model’s generated response claims are coming from.  
They help users gain visibility into the model reasoning, and sanity check the final model generation.  
These citations are optional — you can decide to ignore them.


In [43]:
print("Citations that support the final answer:")
for cite in response.citations:
  print(cite)

Citations that support the final answer:
{'start': 18, 'end': 37, 'text': 'most powerful model', 'document_ids': ['doc_0', 'doc_1', 'doc_2']}
{'start': 45, 'end': 62, 'text': 'Command R family.', 'document_ids': ['doc_0', 'doc_1', 'doc_2']}
{'start': 75, 'end': 94, 'text': 'enhanced assistance', 'document_ids': ['doc_1']}
{'start': 98, 'end': 106, 'text': 'startups', 'document_ids': ['doc_1']}
{'start': 128, 'end': 139, 'text': 'language AI', 'document_ids': ['doc_1']}
{'start': 209, 'end': 219, 'text': 'input cost', 'document_ids': ['doc_0']}
{'start': 223, 'end': 238, 'text': '$3.00/1M tokens', 'document_ids': ['doc_0']}
{'start': 252, 'end': 263, 'text': 'output cost', 'document_ids': ['doc_0']}
{'start': 267, 'end': 284, 'text': '$15.00/1M tokens.', 'document_ids': ['doc_0']}
{'start': 323, 'end': 330, 'text': 'rerank.', 'document_ids': ['doc_2']}
{'start': 334, 'end': 359, 'text': 'boosts the search quality', 'document_ids': ['doc_2']}
{'start': 367, 'end': 374, 'text': 'keyword',

In [44]:
def insert_citations_in_order(text, citations):
    """
    A helper function to pretty print citations.
    """
    offset = 0
    document_id_to_number = {}
    citation_number = 0
    modified_citations = []

    # Process citations, assigning numbers based on unique document_ids
    for citation in citations:
        citation_numbers = []
        for document_id in sorted(citation["document_ids"]):
            if document_id not in document_id_to_number:
                citation_number += 1  # Increment for a new document_id
                document_id_to_number[document_id] = citation_number
            citation_numbers.append(document_id_to_number[document_id])

        # Adjust start/end with offset
        start, end = citation['start'] + offset, citation['end'] + offset
        placeholder = ''.join([f'[{number}]' for number in citation_numbers])
        # Bold the cited text and append the placeholder
        modification = f'**{text[start:end]}**{placeholder}'
        # Replace the cited text with its bolded version + placeholder
        text = text[:start] + modification + text[end:]
        # Update the offset for subsequent replacements
        offset += len(modification) - (end - start)

    # Prepare citations for listing at the bottom, ensuring unique document_ids are listed once
    unique_citations = {number: doc_id for doc_id, number in document_id_to_number.items()}
    citation_list = '\n'.join([f'[{doc_id}] source: "{documents[doc_id - 1]["snippet"]}"' for doc_id, number in sorted(unique_citations.items(), key=lambda item: item[1])])
    text_with_citations = f'{text}\n\n{citation_list}'

    return text_with_citations


print(insert_citations_in_order(response.text, response.citations))


Command R+ is the **most powerful model**[1][2][3] in the **Command R family.**[1][2][3] It provides **enhanced assistance**[2] to **startups**[2] looking to implement **language AI**[2] in their operations.

Compared to the default model, it has a higher **input cost**[1] of **$3.00/1M tokens**[1] and a higher **output cost**[1] of **$15.00/1M tokens.**[1]

Command R+ also has a feature called **rerank.**[3] It **boosts the search quality**[3] of any **keyword**[3] or **vector search system**[3] without requiring any **system overhaul**[3] or **replacement.**[3]

[1] source: "Introducing Command R+: Our new, most powerful model in the Command R family.
Try now
Access our models directly through our API and pay for what you use, or deploy on Amazon SageMaker.
Get free, rate-limited usage for learning and prototyping. Usage is free until you go into production
Power your business with hosted language models.
Features
Default Model
Input
Output
Command R+
Input
$3.00
/1M tokens
Output
$1